In [ ]:
import requests
import json
import pandas as pd

api_key = '1ZxcVL8gBZIFARcf1f9lOR4SZsK1lTSF'

ticker = 'ASME.AS'
data_type = 'historical-price-full'

url = f"https://financialmodelingprep.com/api/v3/{data_type}/{ticker}?apikey={api_key}"

r = requests.get(url)
data = r.json()

In [ ]:
list(data.keys()) != ['symbol','historical']
url

In [ ]:
data.keys()

In [ ]:
def request_is_good(json_r):
    b_return = False
    req_keys = list(json_r.keys())
    if req_keys == ['symbol','historical']:
        b_return = True

    return b_return

In [ ]:
request_is_good(data)

In [ ]:
import os
pp='c:\\Users\\top kek\\Desktop\\Python\\2_External APIs\\market data\\'
os.listdir(pp)

In [ ]:
path = 'c:\\Users\\top kek\\Desktop\\Python\\2_External APIs\\market data\\'
ticker = "ADBE"
df_temp = pd.read_csv(path + ticker + '.csv')
df_temp = df_temp.set_index('date')
df_temp = df_temp[['adjClose']]
df_temp.rename(columns={'adjClose': ticker}, inplace=True)
market_data = df_temp

print(market_data)

In [ ]:
import scipy.stats as stats
import seaborn as sns
rvs = stats.norm(loc=0, scale=1)
normal_sample = rvs.rvs(size=1000)
sns.set()
sns.distplot(normal_sample)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df_logreturns = market_data.apply(np.log)-market_data.shift(1).apply(np.log)
df_logreturns.dropna(inplace=True)
real_sample = np.array(df_logreturns['ADBE'])

stats.probplot(real_sample, dist="norm", plot=plt)

In [ ]:
# Step 1: Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import seaborn as sns

# Step 2: Generate or load your data
# For example, let's generate a sample dataset
data = np.random.normal(0, 1, 1000)

# Step 3: Calculate skewness and kurtosis
data_skewness = skew(data)
data_kurtosis = kurtosis(data, fisher=False)  # Use Pearson's definition of kurtosis

# Step 4: Plot the Cullen and Frey graph
plt.figure(figsize=(10, 8))

# Plot the point for the data
plt.scatter(data_kurtosis, data_skewness**2, color='blue', label='Data', zorder=5)

# Label axes
plt.xlabel('Kurtosis')
plt.ylabel('Squared Skewness')
plt.title('Cullen and Frey Graph')

# Plot theoretical distributions
# Normal distribution
plt.scatter(3, 0, color='green', label='Normal Distribution', zorder=5)

# Exponential distribution
exp_kurtosis = 9
exp_skewness = 2**2
plt.scatter(exp_kurtosis, exp_skewness, color='red', label='Exponential Distribution', zorder=5)

# Beta distribution (skewed)
beta_skewness = np.linspace(0, 4, 100)
beta_kurtosis = 3 + beta_skewness
plt.plot(beta_kurtosis, beta_skewness, label='Beta Distribution', color='orange')

# Gamma distribution
gamma_skewness = np.linspace(0, 5, 100)
gamma_kurtosis = 3 + 6 / (gamma_skewness**2)
plt.plot(gamma_kurtosis, gamma_skewness**2, label='Gamma Distribution', color='purple')

# Chi-squared distribution
chi2_kurtosis = 3 + 12 / (gamma_skewness)
chi2_skewness = 2 / np.sqrt(gamma_skewness)
plt.plot(chi2_kurtosis, chi2_skewness**2, label='Chi-squared Distribution', color='brown')

# Add legend
plt.legend()
plt.grid(True)

# Show plot
plt.show()
plt.savefig('Cullen Frey Graph.png')

In [ ]:
# Step 1: Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, kendalltau, spearmanr, rankdata
from statsmodels.distributions.empirical_distribution import ECDF
from copulas.multivariate import GaussianMultivariate
from copulas.visualization import scatter_2d

# Step 2: Generate or load data
# For example, let's create synthetic data with a known dependence structure
np.random.seed(42)
n_samples = 1000

# Generate data from two correlated normal distributions
data_x = np.random.normal(0, 1, n_samples)
data_y = 0.5 * data_x + np.random.normal(0, 1, n_samples)

data = pd.DataFrame({'X': data_x, 'Y': data_y})

# Step 3: Transform marginals to uniform using empirical CDF
ecdf_x = ECDF(data['X'])
ecdf_y = ECDF(data['Y'])

u_x = ecdf_x(data['X'])
u_y = ecdf_y(data['Y'])

u_data = pd.DataFrame({'U_X': u_x, 'U_Y': u_y})

# Step 4: Fit a Gaussian copula to the transformed data
copula = GaussianMultivariate()
copula.fit(u_data)

# Step 5: Generate samples from the fitted copula
copula_samples = copula.sample(n_samples)

# Step 6: Visualize the results
plt.figure(figsize=(14, 6))

# Original data
plt.subplot(1, 2, 1)
plt.scatter(data['X'], data['Y'], alpha=0.5)
plt.title('Original Data')
plt.xlabel('X')
plt.ylabel('Y')

# Samples from copula
plt.subplot(1, 2, 2)
scatter_2d(copula_samples, title='Samples from Copula')

plt.show()


In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model

# Step 2: Load and prepare financial data
# For example, let's use some synthetic data
np.random.seed(42)
n_days = 1000
returns = np.random.normal(0, 1, n_days)
volatility = np.zeros(n_days)

# Simulate some volatility clustering
for i in range(1, n_days):
    returns[i] += 0.9 * returns[i-1] * np.exp(-returns[i-1]**2)

# Convert to a DataFrame
dates = pd.date_range(start='2020-01-01', periods=n_days)
data = pd.DataFrame({'Returns': returns}, index=dates)

# Step 3: Plot the time series and returns
plt.figure(figsize=(14, 6))
plt.plot(data.index, data['Returns'], label='Returns')
plt.title('Simulated Financial Returns')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.legend()
plt.show()

# Step 4: Estimate volatility using GARCH model
# Fit a GARCH(1, 1) model
garch_model = arch_model(data['Returns'], vol='Garch', p=1, q=1)
garch_fit = garch_model.fit(disp='off')

# Step 5: Visualize the volatility clustering
data['Volatility'] = garch_fit.conditional_volatility

plt.figure(figsize=(14, 6))
plt.plot(data.index, data['Returns'], label='Returns', alpha=0.5)
plt.plot(data.index, data['Volatility'], label='Estimated Volatility', color='red')
plt.title('Volatility Clustering in Financial Returns')
plt.xlabel('Date')
plt.ylabel('Returns / Volatility')
plt.legend()
plt.show()

In [ ]:
from ReturnProfile import asset_performance
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

eq = asset_performance("ADSK")

min_x = min(eq.real_sample)
max_x = max(eq.real_sample)
x_theo = np.linspace(min_x,max_x,90)

plt.plot(stats.norm.cdf(x_theo, eq.mean_log_return, eq.vola))




In [ ]:
import pandas as pd
from ReturnProfile import asset_performance
eq = asset_performance("ADSK")

In [ ]:
df_vwap = pd.merge(eq.df_prices,eq.df_volumes,left_index=True, right_index=True, how='left')
df_vwap.columns = ['p','q']
n = eq.dict_quick_stats['observation period (days)']
vwap_range = 30
df_vwap['vwap'] = None
# li_vwap = []

for i in range(1,n-vwap_range):
            
    df_temp = df_vwap.iloc[i:i+vwap_range]
    df_temp['weights'] = df_temp.q / df_temp.q.sum()
    vwap = (df_temp.p * df_temp.weights).sum()
    df_vwap['vwap'].iloc[i-1] = vwap

    # li_vwap.append(vwap)

df_vwap

In [ ]:

li_vwap

In [27]:
import pandas as pd
from ReturnProfile import asset_performance
eq = asset_performance("ADBE")
eq.df_stats

{'mean': log_return   -0.001918
dtype: float64, 'vola': log_return    0.014139
dtype: float64, 'skew': log_return   -2.960569
dtype: float64, 'kurt': log_return    17.174896
dtype: float64, 'std': log_return    0.022908
dtype: float64, 'observation period (days)': 96}


c:\Users\ashve\Desktop\Projects\ReturnProfile.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['weights'] = df_temp['volume'] / df_temp['volume'].sum()
c:\Users\ashve\Desktop\Projects\ReturnProfile.py:150: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_ind

,date,close,log_return,rank_asc,rank_des
rank_asc,,,,,
1,2024-03-15,492.46,-0.147012,1,96
2,2024-02-16,546.66,-0.077041,2,95
3,2024-03-05,544.84,-0.041524,3,94
4,2024-02-06,607.14,-0.037754,4,93
5,2024-02-12,611.84,-0.024811,5,92
...,...,...,...,...,...
91,2024-01-08,580.55,0.027858,91,6
92,2024-02-23,553.44,0.029094,92,5
93,2024-01-19,611.55,0.029623,93,4
